In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.express as px
from sklearn.metrics import silhouette_score

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from random import sample
from numpy.random import uniform
from math import isnan

In [2]:
raw_df = pd.read_csv('https://media.githubusercontent.com/media/akilaliyanage/ML_Assignment2_IT19120812_IT19188546_IT19120362/main/Pakistan_Largest_Ecommerce_Dataset.csv',
                         dtype={'CustomerID': str,'InvoiceID': str})

raw_df.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,7,8,9,11,12,13,14,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,...,Month,Customer Since,M-Y,FY,Customer ID,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,211131.0,complete,7/1/2016,kreations_YI 06-L,1950.0,1.0,1950.0,100147443,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,1.0,NaN,NaN,NaN,NaN,NaN
1,211133.0,canceled,7/1/2016,kcc_Buy 2 Frey Air Freshener & Get 1 Kasual Bo...,240.0,1.0,240.0,100147444,Beauty & Grooming,\N,...,7.0,2016-7,7-2016,FY17,2.0,NaN,NaN,NaN,NaN,NaN
2,211134.0,canceled,7/1/2016,Ego_UP0017-999-MR0,2450.0,1.0,2450.0,100147445,Women's Fashion,\N,...,7.0,2016-7,7-2016,FY17,3.0,NaN,NaN,NaN,NaN,NaN
3,211135.0,complete,7/1/2016,kcc_krone deal,360.0,1.0,60.0,100147446,Beauty & Grooming,R-FSD-52352,...,7.0,2016-7,7-2016,FY17,4.0,NaN,NaN,NaN,NaN,NaN
4,211136.0,order_refunded,7/1/2016,BK7010400AG,555.0,2.0,1110.0,100147447,Soghaat,\N,...,7.0,2016-7,7-2016,FY17,5.0,NaN,NaN,NaN,NaN,NaN


In [3]:
raw_df.rename(columns = {'Customer Since':'Customer_Since', 'Customer ID':'Customer_ID' , 'Unnamed: 21' : 'Unnamed:21', 'Unnamed: 22' : 'Unnamed:22', 'Unnamed: 23' : 'Unnamed:23', 'Unnamed: 24' : 'Unnamed:24', 'Unnamed: 25' : 'Unnamed:25'}, inplace = True)

In [4]:
print('Initial Dataset Diamentions:', raw_df.shape)
dist_status =  raw_df.status.unique()
dist_status

Initial Dataset Diamentions: (1048575, 26)


array(['complete', 'canceled', 'order_refunded', 'received', 'refund',
       'closed', 'fraud', 'holded', 'exchange', 'pending_paypal', 'paid',
       '\\N', 'cod', 'pending', nan, 'processing', 'payment_review'],
      dtype=object)

In [5]:
print('Number Of Distinct Order Status :', len(dist_status))

Number Of Distinct Order Status : 17


In [6]:
dist_cate = raw_df.category_name_1.unique()
dist_cate

array(["Women's Fashion", 'Beauty & Grooming', 'Soghaat',
       'Mobiles & Tablets', 'Appliances', 'Home & Living',
       "Men's Fashion", 'Kids & Baby', '\\N', 'Others', 'Entertainment',
       'Computing', 'Superstore', 'Health & Sports', 'Books',
       'School & Education', nan], dtype=object)

In [7]:
print('Number Of Distinct Customers :', len(raw_df.Customer_ID.unique()))

Number Of Distinct Customers : 115327


In [8]:
raw_df_stats=pd.DataFrame(raw_df.dtypes).T.rename(index={0:'Column Data Type'})
raw_df_stats=raw_df_stats.append(pd.DataFrame(raw_df.isnull().sum()).T.rename(index={0:'Null Value Count'}))
raw_df_stats=raw_df_stats.append(pd.DataFrame(raw_df.isnull().sum()/raw_df.shape[0]*100).T.rename(index={0:'Null Value Percentage'}))

display(raw_df_stats)

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,...,Month,Customer_Since,M-Y,FY,Customer_ID,Unnamed:21,Unnamed:22,Unnamed:23,Unnamed:24,Unnamed:25
Column Data Type,float64,object,object,object,float64,float64,float64,object,object,object,...,float64,object,object,object,float64,float64,float64,float64,float64,float64
Null Value Count,464051,464066,464051,464071,464051,464051,464051,464051,464215,601226,...,464051,464062,464051,464051,464062,1048575,1048575,1048575,1048575,1048575
Null Value Percentage,44.255394,44.256825,44.255394,44.257302,44.255394,44.255394,44.255394,44.255394,44.271034,57.337434,...,44.255394,44.256443,44.255394,44.255394,44.256443,100.0,100.0,100.0,100.0,100.0


In [9]:
stage1_df = raw_df.drop(raw_df[(raw_df.Customer_ID.isnull()) | (raw_df.status.isnull()) | (raw_df.category_name_1.isnull())].index)
stage1_df = stage1_df.drop(stage1_df[(stage1_df.status == 'canceled') | (stage1_df.status == 'order_refunded') | (stage1_df.status == 'refund') | (stage1_df.status == '\\N') | (stage1_df.status == 'fraud') | (stage1_df.status == 'cod') | (stage1_df.category_name_1 == '\\N')].index)
stage1_df = stage1_df.drop(['Unnamed:21', 'Unnamed:22' , 'Unnamed:23', 'Unnamed:24' , 'Unnamed:25'], axis=1)

In [10]:
stage1_df_stats=pd.DataFrame(stage1_df.dtypes).T.rename(index={0:'Column Data Type'})
stage1_df_stats=stage1_df_stats.append(pd.DataFrame(stage1_df.isnull().sum()).T.rename(index={0:'Null Value Count'}))
stage1_df_stats=stage1_df_stats.append(pd.DataFrame(stage1_df.isnull().sum()/stage1_df.shape[0]*100).T.rename(index={0:'Null Value Percentage'}))

display(stage1_df_stats)

,item_id,status,created_at,sku,price,qty_ordered,grand_total,increment_id,category_name_1,sales_commission_code,...,payment_method,Working Date,BI Status,MV,Year,Month,Customer_Since,M-Y,FY,Customer_ID
Column Data Type,float64,object,object,object,float64,float64,float64,object,object,object,...,object,object,object,object,float64,float64,object,object,object,float64
Null Value Count,0,0,0,0,0,0,0,0,0,63195,...,0,0,0,0,0,0,0,0,0,0
Null Value Percentage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.422574,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
print('Stage 1 Dataframe Diamentions:', stage1_df.shape)

dist_status =  stage1_df.status.unique()
print('Distinct Status : ' , dist_status)
print('Number Of Distinct Order Status :', len(dist_status))

dist_cate = stage1_df.category_name_1.unique()
print('Distinct Categories : ' , dist_cate)
print('Number Of Distinct Categories :', len(dist_cate))

print('Number Of Distinct Customers :', len(stage1_df.Customer_ID.unique()))

Stage 1 Dataframe Diamentions: (309437, 21)
Distinct Status :  ['complete' 'received' 'closed' 'holded' 'exchange' 'pending_paypal'
 'paid' 'pending' 'processing' 'payment_review']
Number Of Distinct Order Status : 10
Distinct Categories :  ["Women's Fashion" 'Beauty & Grooming' 'Soghaat' 'Appliances'
 'Home & Living' 'Kids & Baby' 'Others' 'Mobiles & Tablets'
 "Men's Fashion" 'Computing' 'Superstore' 'Health & Sports' 'Books'
 'School & Education' 'Entertainment']
Number Of Distinct Categories : 15
Number Of Distinct Customers : 78998


In [ ]:
stage1_df.info()

In [ ]:
stage1_df.describe()

In [ ]:
per_customer_grand_total_df = stage1_df[['Customer_ID', 'grand_total']].groupby('Customer_ID', as_index = False).sum()
per_customer_grand_total_df.rename(columns = {'grand_total' : 'Monetary'}, inplace = True)
per_customer_grand_total_df

In [ ]:
per_customer_shopping_frequency_df =  stage1_df[['Customer_ID', 'created_at']].groupby('Customer_ID', as_index = False).count()
per_customer_shopping_frequency_df.rename(columns = {'created_at': 'Frequency'}, inplace = True)
per_customer_shopping_frequency_df

In [ ]:
processed_sales_df = per_customer_grand_total_df.merge(per_customer_shopping_frequency_df, on = "Customer_ID", how = "inner")
processed_sales_df

In [ ]:
most_reason_order_df = stage1_df[['Customer_ID', 'created_at']].groupby('Customer_ID', as_index = False).max()
most_reason_order_df.rename(columns = {'created_at': 'Most_Reason_Order_Date'}, inplace = True)
most_reason_order_df

In [ ]:
stage1_df['created_at1'] = pd.to_datetime(stage1_df['created_at'])
most_reason_order_df['Most_Reason_Order_Date1'] = pd.to_datetime(most_reason_order_df['Most_Reason_Order_Date'])